In [ ]:
!apt install python3.12

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.12-stdlib
Suggested packages:
  python3.12-venv
The following NEW packages will be installed:
  libpython3.12-stdlib python3.12
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 5,415 kB of archives.
After this operation, 22.5 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.12-stdlib amd64 3.12.8-1+jammy1 [2,889 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.12 amd64 3.12.8-1+jammy1 [2,526 kB]
Fetched 5,415 kB in 2s (2,689 kB/s)
Selecting previously unselected package libpython3.12-stdlib:amd64.
(Reading database ... 124373 files and directories currently installed.)
Preparing to unpack .../libpython3.12-stdlib_3.12.8-1+jammy1_amd64.deb ...
Unpacking libpython3.12-stdlib:amd64

In [ ]:
!pip install pqcrypto

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pqcrypto: filename=pqcrypto-0.1.3-cp310-cp310-manylinux_2_35_x86_64.whl size=2119082 sha256=471e284861276cdb8d65ba2f7a65b0c506a26ddaa8ff575a5aab34652bd42f26
  Stored in directory: /root/.cache/pip/wheels/89/13/4f/b2c678a9f6c473acb0d259d47c30387bab4f836f28dab19f1c
Successfully built pqcrypto


In [2]:
!pip install argon2-cffi pycryptodome


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.7 MB/s eta 0:00:00


In [3]:
!pip install twofish

  Preparing metadata (setup.py) ... done
  Created wheel for twofish: filename=twofish-0.3.0-cp310-cp310-linux_x86_64.whl size=24196 sha256=f7b6228efec09b34a14e19c1a692728c2c2450d12bc5736169234dff2ed5ec1c
  Stored in directory: /root/.cache/pip/wheels/a7/3c/27/c390be4f3e8a299d4b2836f8daa19697eb991eacbfabe25031
Successfully built twofish


In [31]:
import os
from argon2 import PasswordHasher
from twofish import Twofish


def extract_hash(argon2_hash_string):
    """Extract the hash value from an Argon2 hash string."""
    try:
        parts = argon2_hash_string.split('$')
        if len(parts) < 6:
            raise ValueError("Invalid Argon2 hash format")
        return parts[-1]
    except Exception as e:
        print(f"Error extracting hash: {e}")
        return None


class SecureEncryptionSystem:
    def __init__(self):
        self.ph = PasswordHasher()

    def hash_password(self, password):
        """Hash the user-provided password using Argon2."""
        hashed = self.ph.hash(password)
        return hashed

    def _pad(self, data):
        """Pad data to a multiple of 16 bytes."""
        padding_length = 16 - (len(data) % 16)
        return data + bytes([padding_length] * padding_length)

    def _unpad(self, data):
        """Remove padding from data."""
        padding_length = data[-1]
        return data[:-padding_length]

    def _xor_whitening(self, data, whitening_keys):
        """Apply XOR whitening to the data using the whitening keys."""
        return bytes(d ^ k for d, k in zip(data, whitening_keys))

    def _reverse_xor_whitening(self, data, whitening_keys):
        """Reverse XOR whitening."""
        return bytes(d ^ k for d, k in zip(data, whitening_keys))

    def encrypt_with_twofish(self, key, plaintext):
        """Encrypt the plaintext using the Twofish encryption algorithm with whitening."""
        tf = Twofish(key[:16])  # Twofish accepts 16-byte keys
        padded_plaintext = self._pad(plaintext.encode())
        whitening_keys = key[:16]
        whitened_plaintext = self._xor_whitening(padded_plaintext, whitening_keys * (len(padded_plaintext) // 16 + 1))
        ciphertext = b''
        for i in range(0, len(whitened_plaintext), 16):
            block = whitened_plaintext[i:i + 16]
            ciphertext += tf.encrypt(block)
        return ciphertext

    def decrypt_with_twofish(self, key, ciphertext):
        """Decrypt the ciphertext using the Twofish encryption algorithm with whitening."""
        tf = Twofish(key[:16])  # Twofish accepts 16-byte keys
        whitening_keys = key[:16]
        decrypted_data = b''
        for i in range(0, len(ciphertext), 16):
            block = ciphertext[i:i + 16]
            decrypted_data += tf.decrypt(block)
        original_plaintext = self._reverse_xor_whitening(decrypted_data, whitening_keys * (len(decrypted_data) // 16 + 1))
        return self._unpad(original_plaintext).decode()

    def verify_hash(self, password, stored_hash):
        """Verify the provided password matches the stored hash."""
        try:
            if self.ph.verify(stored_hash, password):
                return True
        except Exception as e:
            return False

    def brute_force_attack(self, stored_hash, max_password_length=4):
      possible_characters = string.ascii_letters + string.digits + string.punctuation
      for password_length in range(1, max_password_length + 1):
        for password in itertools.product(possible_characters, repeat=password_length):
          password = ''.join(password)
          hashed_password = self.ph.hash(password)
          print(extract_hash(hashed_password))
          if hashed_password == stored_hash:
            return password

      return None


# Example Usage
if __name__ == "__main__":
    password = "123"
    system = SecureEncryptionSystem()
    hashed_password = system.hash_password(password)
    print(f"Hashed Password: {extract_hash(hashed_password)}")
    encryption_key = extract_hash(hashed_password).encode()[:16]
    print(encryption_key.decode())
    ciphertext = system.encrypt_with_twofish(encryption_key, hashed_password)
    print(f"Ciphertext: {ciphertext.hex()}")
    decrypted_data = system.decrypt_with_twofish(encryption_key, ciphertext)
    print(f"Decrypted Hashed Password: {decrypted_data}")
    hash_match = system.verify_hash(password, hashed_password)
    if hash_match:
        print("Hash Matched: The password is valid.")
    else:
        print("Hash Mismatch: The password is invalid.")


    cracked_password = system.brute_force_attack(hashed_password)
    if cracked_password:
        print(f"Cracked Password: {cracked_password}")
    else:
        print("Failed to crack the password.")

Hashed Password: e1YwdqgZLIeklYjwmn1Bhw/6yKwV/YeFpz+1WiPKrQw
e1YwdqgZLIeklYjw
Ciphertext: 52ffcac409e037e7dc2cd137c6079fca53ac0bf2b8aa5f34edc26e1789750c54cdf8afd26fd414de731a2966fbadbf3474ed08270398a57066d708abdd37c592a301df9389e95f2924ee71772ff4dd1f89d903070d86ea704e3c3ce7b9c4587adcd2b70137f4c1e9315e7107885b232b
Decrypted Hashed Password: $argon2id$v=19$m=65536,t=3,p=4$Q1Hbr6zfxxTMGuCmaJDxjQ$e1YwdqgZLIeklYjwmn1Bhw/6yKwV/YeFpz+1WiPKrQw
Hash Matched: The password is valid.
...


KeyboardInterrupt: 